In [2]:
import requests
from bs4 import BeautifulSoup
import unicodedata

def scrape_real_estate(today, yesterday, url):
    # Define headers to simulate a browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com",
        'accept': 'application/json',
        'Content-Type': 'application/json',
        "DNT": "1",
        "Jeff": "Lango-alas"
    }
    
    # Request the webpage content
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    property_listings = soup.find_all('div', class_="wp-block property list")

    # Initialize dictionary to store data
    data = {
        "house_type": [],
        "house_address": [],
        "date_listed": [],
        "price": [],
        "house_desc": []
    }

    # Iterate over each property listing
    for listing in property_listings:
    
        # Extract house type
        house_type = listing.find("h4", class_="content-title").text
        if "bedroom" in house_type:
            data["house_type"].append(house_type)        

            # Extract house address
            raw_address = listing.find("address", class_="voffset-bottom-10").text
            normalized_address = unicodedata.normalize("NFKD", raw_address).strip()
            data["house_address"].append(normalized_address)

            # Extract date listed
            date_listed_span = listing.find("span", class_="added-on").text.split()[2:]
            recently_added_span = listing.find("span", class_="added-on added-recently")

            if not date_listed_span:
                if recently_added_span:
                    if "Today" in recently_added_span.text:
                        listing_date = today
                    else:
                        listing_date = yesterday
                    data["date_listed"].append(listing_date)
            else:
                listing_date = " ".join(date_listed_span)
                data["date_listed"].append(listing_date)

            # Extract price
            price = listing.select('.price')[1].text
            data["price"].append(price)

            # Extract house description
            description = listing.find("ul", class_="aux-info").text[:-12]
            data["house_desc"].append(description)
    
    return data


In [ ]:
import random
import time
import numpy as np

# Generate an array of values to check for when to pause execution
pause_checkpoints = np.arange(30, 4000, 30)
page_number = 2
scraped_data = []

# Initial URL for scraping
base_url = "https://nigeriapropertycentre.com/for-sale/imo?page=2"
current_url = base_url

while page_number <= 19:
    # Scrape real estate data
    data = scrape_real_estate("21 Nov 2022", "20 Nov 2022", url=current_url)
    scraped_data.append(data)
    
    # Check if the current page number is in the pause checkpoints
    if page_number in pause_checkpoints:
        pause_duration = random.randint(15, 40)
        print(f"Pausing at {current_url} for {pause_duration} seconds.")
        time.sleep(pause_duration)
    
    # Update URL to the next page
    current_url = base_url[:-1] + str(page_number + 1)
    page_number += 1


In [ ]:
df = pd.DataFrame()
for item in scraped_data:
    df = pd.concat([df, pd.DataFrame(item)], axis=0)